#### Word2Vec
- 단어 간의 유사성 측정하기 위해 분포 가설을 기반 개발
    - 분포 가설: 같은 문맥에서 함께 자주 나타나는 단어는 서로 유사한 의미를 가질 가능성이 높다는 가정
        - 동시 발생 확률 분포를 이용해 단어 간의 유사성 측정
- 유사한 문맥에서 등장하는 단어는 비슷한 벡터 공강산 위치를 갖게 됨

#### 단어 벡터화
- 방법
    - 희소 표현: 데이터를 고차원의 벡터로 표현 벡터 대부분 요소가 0으로 채워짐 
        - ex) 어휘 사전의 크기가 10,000이고, 특정 문서에 100개의 고유 단어만 포함되어 있다면, 이 문서는 100개의 비-제로 값과 9,900개의 제로 값을 가진 10,000차원 벡터로 표현
        - BoW: 순서에 관계없이 단어의 출현 빈도로만 표현하는 것
        - 원-핫 인코딩: 각 단어를 고유한 색인 값으로 매핑
        - TF-IDF: 단어의 빈도수와 함께 단어가 전체 문서 집합에서 얼마나 고유한지를 함께 고려하는 방법
        - 장점: 명확한 관계를 제공
        - 단점: 고차원이며, 대부분의 값이 0으로 채워져 있어 메모리와 계산 효율이 낮고 단어의 의미적 관계를 포착하지 못하는 경우가 많음
    - 밀집 표현
        - Word2Vec
        - 장점: 의미적 유사성을 기반으로 한 단어 간의 관계를 잘 포착, 계산 효율이 높음
        - 단점: 특징 차원이 구체적으로 어떤 의미를 가지는지 해석하기 어려울 수 있음

    

#### CBoW
- 주변에 단어를 가지고 중간에 있단 단어 예측하는 방법
- 중심 단어(Center Word): 예측할 단어
- 주변 단어(Context Word): 예측에 사용되는 단어
- sliding window를 이용해 이동해 가며 학습 -> 한 번의 학습으로 여러 개의 중심 단어, 그에 대한 주변 단어 학습


#### Skip-gram
- 중심 단어를 입력으로 주변 단어를 예측하는 모델